In [1]:
import copy
import numpy as np
import open3d as o3d
import pickle
import scipy.spatial

import sys
sys.path.insert(1, '../src/')
import metrics
from plane_env_utils import generate_planes

In [2]:
def trajectory_perturbation(Ts, cov=0.1):
    Ts_noise = []
    for T in Ts:
        T_noised = copy.deepcopy(T)
        T_noised[:3, 3] += [np.random.normal(0, cov), np.random.normal(0, cov), 
                            np.random.normal(0, cov)]
        Ts_noise.append(T_noised)
    return Ts_noise

def get_map(pcs, Ts):
    pc_map = o3d.geometry.PointCloud()
    for i, pc in enumerate(pcs):
        pc_map += copy.deepcopy(pc).transform(Ts[i])
        
    return pc_map

def sampling_pipeline(pcs, T_gt, nr_metrics, fr_metric, map_tips=None):
    cov_scaler = 20
    N_samples = 40

    nrs = [[] for _ in range(len(nr_metrics))]
    fr = []
    for i in range(cov_scaler):
        for j in range(N_samples):
            T_pert = trajectory_perturbation(T_gt, cov = 0.02 * (i + 1) / len(pcs))
            pc_map = get_map(pcs, T_pert)
            fr.append(fr_metric(T_gt, T_pert))
            for metric_id, metric in enumerate(nr_metrics):
                nrs[metric_id].append(metric(pc_map, map_tips))

    return nrs, fr

In [ ]:
%%time
def generate_stat(i):
    N_poses = 5
    T_gt = [np.eye(4) for _ in range(N_poses)]

    pcs, orth = generate_planes(N_poses)
    tips = {}
    tips['orth_list'] = orth

    tips['orth_normals'] = []
    for plane in orth:
        pc = o3d.geometry.PointCloud()
        pc.points = o3d.utility.Vector3dVector(plane)
        pc.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=1, max_nn=30))
        tips['orth_normals'].append(np.asarray(pc.normals))

    nr_metrics = [metrics.mean_map_entropy, metrics.mean_plane_variance, 
                  metrics.orth_mme, metrics.orth_mpv]
    nrs, fr = sampling_pipeline(pcs, T_gt, nr_metrics, metrics.rpe, tips)
    save_dict = {}
    save_dict['nrs'] = nrs
    save_dict['fr'] = fr

    with open(str(i) + '-5-density-r.pkl', 'wb') as sfile:
        pickle.dump(save_dict, sfile)

    with open(str(i) + '-5-density-r-pcs.pkl', 'wb') as sfile_1:
        pickle.dump(np.asarray(get_map(pcs, T_gt).points), sfile_1)

    with open(str(i) + '-5-density-r-orth.pkl', 'wb') as sfile_2:
        pickle.dump(orth, sfile_2)

from multiprocessing import Pool

with Pool() as pool:
    pool.map(generate_stat, np.arange(0, 10))